In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [2]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
dataset = datasets.load_dataset(
    "json", 
    data_files={
        "train": "data/train.jsonl",
        "valid": "data/val.jsonl",
        "test": "data/test.jsonl",
    }
)

Found cached dataset json (/home/ben/.cache/huggingface/datasets/json/default-796bb43aecc0ebef/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29001
    })
    valid: Dataset({
        features: ['en', 'de'],
        num_rows: 1015
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [5]:
train_data, valid_data, test_data = dataset["train"], dataset["valid"], dataset["test"]

In [6]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [7]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [8]:
[token.text for token in en_nlp.tokenizer(train_data[0]["en"])]

['Two',
 'young',
 ',',
 'White',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '.']

In [9]:
def tokenize_example(
    example,
    en_nlp,
    de_nlp,
    max_length,
    lower,
    sos_token,
    eos_token
):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

In [10]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp, 
    "de_nlp": de_nlp, 
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}


train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/json/default-796bb43aecc0ebef/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-90410edabe203a13.arrow
Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/json/default-796bb43aecc0ebef/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-488caae24926b262.arrow
Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/json/default-796bb43aecc0ebef/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-c0db50ddc92a9bca.arrow


In [11]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [12]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    "<unk>",
    "<pad>",
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,  
)

In [13]:
en_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'a', '.', 'in', 'the', 'on', 'man']

In [14]:
en_vocab.get_stoi()["the"]

7

In [15]:
en_vocab["the"]

7

In [16]:
len(en_vocab), len(de_vocab)

(5893, 7853)

In [17]:
"the" in en_vocab

True

In [18]:
"The" in en_vocab

False

In [19]:
# en_vocab["The"]

In [20]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [21]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [22]:
en_vocab["The"]

0

In [23]:
en_vocab.get_itos()[0]

'<unk>'

In [24]:
en_vocab.lookup_indices(train_data[0]["en_tokens"])

[2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3]

In [25]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(train_data[0]["en_tokens"]))

['<sos>',
 'two',
 'young',
 ',',
 'white',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '.',
 '<eos>']

In [26]:
def numericalize_data(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [27]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_data, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_data, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_data, fn_kwargs=fn_kwargs)

Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/json/default-796bb43aecc0ebef/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-91dd0e4eba5def85.arrow
Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/json/default-796bb43aecc0ebef/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-f34872e26766857b.arrow
Loading cached processed dataset at /home/ben/.cache/huggingface/datasets/json/default-796bb43aecc0ebef/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-f2bdcc111f8d12e7.arrow


In [28]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

In [29]:
train_data = train_data.with_format(
    type='torch', 
    columns=["en_ids", "de_ids"], 
    output_all_columns=True
)

valid_data = valid_data.with_format(
    type='torch', 
    columns=["en_ids", "de_ids"],
    output_all_columns=True,
)

test_data = test_data.with_format(
    type='torch',
    columns=["en_ids", "de_ids"],
    output_all_columns=True,
)

In [30]:
train_data[0]

{'en_ids': tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 'de_ids': tensor([   2,   18,   26,  253,   30,   84,   20,   88,    7,   15,  110, 7647,
         3171,    4,    3]),
 'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [31]:
def get_collate_fn(pad_index):
    
    def collate_fn(batch):
        batch_en_ids = [i["en_ids"] for i in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = [i["de_ids"] for i in batch]
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch
    
    return collate_fn

In [32]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    
    collate_fn = get_collate_fn(pad_index)
    
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    
    return data_loader

In [33]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [34]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

In [35]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))    
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [36]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hidden_dim == decoder.hidden_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0,:]
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1
        return outputs

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [38]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [39]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [40]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,898,501 trainable parameters


In [41]:
optimizer = optim.Adam(model.parameters())

In [42]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [43]:
def train_fn(model, data_loader, optimizer, criterion, clip, device):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["de_ids"].to(device)
        trg = batch["en_ids"].to(device)
        optimizer.zero_grad()
        output = model(src, trg)        
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [44]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            output = model(src, trg, 0) #turn off teacher forcing
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [45]:
n_epochs = 10
clip = 1.0

best_valid_loss = float('inf')

for epoch in tqdm.tqdm(range(n_epochs)):
        
    train_loss = train_fn(model, train_data_loader, optimizer, criterion, clip, device)
    valid_loss = evaluate_fn(model, valid_data_loader, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid PPL: {np.exp(valid_loss):7.3f}')

 10%|████▍                                       | 1/10 [00:15<02:19, 15.51s/it]

	Train Loss: 5.045 | Train PPL: 155.188
	Valid Loss: 5.088 | Valid PPL: 162.095


 20%|████████▊                                   | 2/10 [00:31<02:05, 15.68s/it]

	Train Loss: 4.485 | Train PPL:  88.687
	Valid Loss: 4.793 | Valid PPL: 120.701


 30%|█████████████▏                              | 3/10 [00:47<01:50, 15.72s/it]

...

	Train Loss: 3.373 | Train PPL:  29.173
	Valid Loss: 4.051 | Valid PPL:  57.471


 90%|███████████████████████████████████████▌    | 9/10 [02:20<00:15, 15.54s/it]

	Train Loss: 3.254 | Train PPL:  25.892
	Valid Loss: 3.997 | Valid PPL:  54.436


100%|███████████████████████████████████████████| 10/10 [02:35<00:00, 15.59s/it]

	Train Loss: 3.118 | Train PPL:  22.602
	Valid Loss: 3.960 | Valid PPL:  52.437


In [46]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |')

| Test Loss: 3.971 | Test PPL:  53.042 |


In [47]:
def translate_sentence(
    sentence, 
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    if isinstance(sentence, str):
        tokens = [token.text for token in de_nlp.tokenizer(sentence)]
    else:
        tokens = [token for token in sentence]
    if lower:
        tokens = [token.lower() for token in tokens]
    tokens = [sos_token] + tokens + [eos_token]
    ids = de_vocab.lookup_indices(tokens)
    tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
    with torch.no_grad():
        hidden, cell = model.encoder(tensor)
    inputs = en_vocab.lookup_indices([sos_token])
    for _ in range(max_output_length):
        inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
        predicted_token = output.argmax(-1).item()
        inputs.append(predicted_token)
        if predicted_token == en_vocab[eos_token]:
            break
    tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [48]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.',
 'A man in an orange hat starring at something.')

In [49]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [50]:
translation

['<sos>',
 'a',
 'man',
 'in',
 'a',
 'white',
 'hat',
 'is',
 'a',
 'a',
 '.',
 '.',
 '<eos>']

In [51]:
translations = [
    translate_sentence(
        example["de"],
        model,
        en_nlp,
        de_nlp,
        en_vocab,
        de_vocab,
        lower,
        sos_token,
        eos_token,
        device,  
    ) for example in tqdm.tqdm(test_data)
]

100%|██████████████████████████████████████| 1000/1000 [00:03<00:00, 255.05it/s]


In [52]:
bleu = evaluate.load("bleu")

In [53]:
predictions = [
    " ".join(translation[1:-1]) for translation in translations
]

references = [
    [example["en"]] for example in test_data
]

In [54]:
predictions[0], references[0]

('a man in a white hat is a a . .',
 ['A man in an orange hat starring at something.'])

In [55]:
def get_tokenizer_fn(nlp, lower):
    
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens
        
    return tokenizer_fn

tokenizer_fn = get_tokenizer_fn(en_nlp, lower)

tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])

(['a', 'man', 'in', 'a', 'white', 'hat', 'is', 'a', 'a', '.', '.'],
 ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.'])

In [56]:
results = bleu.compute(predictions=predictions, references=references, tokenizer=tokenizer_fn)

In [57]:
results

{'bleu': 0.12709809991094775,
 'precisions': [0.46526795536696525,
  0.1804536926488146,
  0.084094723102741,
  0.04102405922270204],
 'brevity_penalty': 0.974249037095902,
 'length_ratio': 0.9745749731965079,
 'translation_length': 12726,
 'reference_length': 13058}